# Simple Model Example

There are two ways of running a simple eCalc™ model:

1. Specifying a YAML model
2. Specifying a Python model

Here we will look at both scenarios:

## The Simple Model
The Simple used in this example is the reference case in [eCalc™ Docs - Simple Model Example](https://equinor.github.io/ecalc/docs/about/modelling/examples/simple).

The model consists of a single installation with:
- Flare
- Gas Export Compressor
- Generator set A:
    - Base Production Load
    - Gas Injection Compressor
    - Produced Water Re-injection Pump
    - Seawater Injection Pump

The model.yaml file contains references to:
- production_data.csv
- genset.csv
- compressor_sampled.csv
- compressor_sampled_with_turbine.csv
- pump_sampled.csv
- pump_chart.csv

## Yaml Model
A YAML model consists of a main YAML file and may reference to other input files - usually CSV-files for timeseries inputs.
For more details about creating a model in YAML, see the [eCalc™ Docs](https://equinor.github.io/ecalc/docs/about)

All files for the example can be found in the directory references in the code below.

Here is how you load and run a YAML-model in eCalc™

In [ ]:
from pathlib import Path
from libecalc.core.graph_result import GraphResult
from libecalc.core.ecalc import EnergyCalculator
from libecalc.common.time_utils import Frequency
from libecalc.input.model import YamlModel
from libecalc.examples import simple
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
model_path = Path(simple.__file__).parent / "model.yaml"
yaml_model = YamlModel(path=model_path, output_frequency=Frequency.NONE)

model = EnergyCalculator(graph=yaml_model.graph)

## Compute

In [ ]:
consumer_results = model.evaluate_energy_usage(yaml_model.variables)
emission_results = model.evaluate_emissions(
    variables_map=yaml_model.variables,
    consumer_results=consumer_results,
)

## Extract

In [ ]:
result = GraphResult(
    graph=yaml_model.graph,
    consumer_results=consumer_results,
    variables_map=yaml_model.variables,
    emission_results=emission_results,
)

## Plot results per component

In [ ]:
def mfig(nax, **kwargs):
    """Create superimposed, shifted, colored axes."""
    fig = plt.figure(num=kwargs.pop('num', None), figsize=kwargs.pop('figsize', None))
    fig.clear()
    fig, ax0 = plt.subplots(num=fig.get_label(), nrows=1, ncols=1)
    fig.subplots_adjust(right=1 - .05*nax)

    axs = []
    for i in range(nax):
        ax = ax0.twinx()
        color=f"C{i}"
        ax.spines.right.set_position(("axes", 1 + .05 * i))
        ax.yaxis.label.set_color(color)
        ax.tick_params(axis='y', colors=color)
        axs.append(ax)
    ax0.set_yticks([])
    return fig, axs


def plot_results(kind, legend_kws={}):
    """Plot `result` components together."""

    # The two data structures are slightly different
    if "emis" in kind.lower():
        kind = "Emissions"
        results = result.emission_results
        def get_data(id_hash):
            r = results[id_hash]
            assert len(r) == 1
            return r['co2'].rate
    else:
        kind = "Generic"
        results = result.consumer_results
        def get_data(id_hash):
            r = results[id_hash]
            return r.component_result.energy_usage

    # Filter `components` by `results`
    components = {id_hash: comp for (id_hash, comp)
                  in yaml_model.graph.components.items()
                  if id_hash in results}

    # Plot
    fig, axs = mfig(len(components), num=kind, figsize=(8, 5))
    fig.suptitle(kind)

    for i, (id_hash, component) in enumerate(components.items()):
        data = get_data(id_hash)
        lbl = f"{component.name}\n({type(component).__name__}, {data.unit})"
        ds = pd.Series(data.values, index=data.timesteps)
        ax = axs[i]
        color = colors[id_hash]
        ds.plot(ax=axs[i], color=color, lw=2, marker=".", xlabel="time", label=lbl)
        ax.yaxis.label.set_color(color)
        ax.tick_params(axis='y', colors=color)

    fig.legend(**{"loc":"lower center", "bbox_to_anchor":(.5, 0),
                  "fontsize": "small", "bbox_transform": axs[0].transAxes,
                  **legend_kws})
    plt.tight_layout()
    return fig

In [ ]:
if __name__ == "__main__":
    plt.ion();
    colors = {id_hash: f'C{i}' for i, id_hash in enumerate(yaml_model.graph.components)}
    fig = plot_results("Generic")
    fig = plot_results("Emissions")

In [ ]:
# Dummy test in order to test to assert that this notebook runs in GitHub Actions
def test_notebook_works():
    assert True